In [1]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth','koi_prad','koi_teq','koi_insol','koi_model_snr','koi_steff','koi_slogg','koi_srad','ra','dec','koi_kepmag']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
y = df['koi_disposition']

In [5]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
X_train.head()

,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,6.768901,133.077240,0.150,3.61600,123.1,1.24,1017,253.30,10.8,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0.733726,132.020050,0.291,2.30900,114.6,0.86,1867,2891.64,13.8,5855,4.578,0.797,284.50391,42.463860,15.770
2879,7.652707,134.460380,0.970,79.89690,641.1,3.21,989,226.81,254.3,6328,4.481,0.963,295.50211,38.983540,13.099
107,7.953547,174.662240,0.300,2.63120,875.4,2.25,696,55.37,38.4,4768,4.536,0.779,291.15878,40.750271,15.660
29,4.959319,172.258529,0.831,2.22739,9802.0,12.21,1103,349.40,696.5,5712,4.359,1.082,292.16705,48.727589,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [7]:
# Scale your data
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

C:\Users\Gonzalo\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\Gonzalo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [9]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [10]:
from tensorflow.keras.utils import to_categorical
# One-hot encoding
#y_train_categorical = to_categorical(y_train)
#y_test_categorical = to_categorical(y_test)

# Train the Model



In [11]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=50, activation='relu', input_dim=15))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                800       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 953
Trainable params: 953
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
5243/5243 - 1s - loss: 0.9266 - acc: 0.5400
Epoch 2/60
5243/5243 - 0s - loss: 0.8202 - acc: 0.6185
Epoch 3/60
5243/5243 - 0s - loss: 0.7812 - acc: 0.6456
Epoch 4/60
5243/5243 - 0s - loss: 0.7597 - acc: 0.6592
Epoch 5/60
5243/5243 - 0s - loss: 0.7441 - acc: 0.6656
Epoch 6/60
5243/5243 - 0s - loss: 0.7325 - acc: 0.6739
Epoch 7/60
5243/5243 - 0s - loss: 0.7227 - acc: 0.6779
Epoch 8/60
5243/5243 - 0s - loss: 0.7154 - acc: 0.6815
Epoch 9/60
5243/5243 - 0s - loss: 0.7078 - acc: 0.6895
Epoch 10/60
5243/5243 - 0s - loss: 0.7025 - acc: 0.6908
Epoch 11/60
5243/5243 - 0s - loss: 0.6978 - acc: 0.6920
Epoch 12/60
5243/5243 - 0s - loss: 0.6950 - acc: 0.6943
Epoch 13/60
5243/5243 - 0s - loss: 0.6896 - acc: 0.6985
Epoch 14/60
5243/5243 - 0s - loss: 0.6865 - acc: 0.6988
Epoch 15/60
5243/5243 - 0s - loss: 0.6837 - acc: 0.6998
Epoch 16/60
5243/5243 - 0s - loss: 0.6791 - acc: 0.6996
Epoch 17/60
5243/5243 - 0s - loss: 0.6762 - acc: 0.7019
Epoch 18/60
5243/5243 - 0s - loss: 0.6744 - acc: 0.7068
E

In [15]:
model_loss, model_accuracy = model.evaluate(
    X_train_scaled, y_train_categorical, verbose=2)
print(
    f"Train Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

5243/5243 - 0s - loss: 0.6174 - acc: 0.7261
Train Neural Network - Loss: 0.617412493630445, Accuracy: 0.7261109948158264


In [16]:
deep_model = Sequential()
deep_model.add(Dense(units=100, activation='relu', input_dim=15))
deep_model.add(Dense(units=100, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               1600      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 12,003
Trainable params: 12,003
Non-trainable params: 0
_________________________________________________________________


In [17]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_test_scaled,
    y_test_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
1748/1748 - 2s - loss: 0.9191 - acc: 0.5515
Epoch 2/100
1748/1748 - 1s - loss: 0.8242 - acc: 0.6121
Epoch 3/100
1748/1748 - 2s - loss: 0.7883 - acc: 0.6402
Epoch 4/100
1748/1748 - 1s - loss: 0.7633 - acc: 0.6585
Epoch 5/100
1748/1748 - 2s - loss: 0.7395 - acc: 0.6693
Epoch 6/100
1748/1748 - 2s - loss: 0.7201 - acc: 0.6882
Epoch 7/100
1748/1748 - 2s - loss: 0.7047 - acc: 0.6836
Epoch 8/100
1748/1748 - 3s - loss: 0.6938 - acc: 0.6974
Epoch 9/100
1748/1748 - 2s - loss: 0.6843 - acc: 0.7025
Epoch 10/100
1748/1748 - 2s - loss: 0.6770 - acc: 0.7008
Epoch 11/100
1748/1748 - 2s - loss: 0.6620 - acc: 0.7082
Epoch 12/100
1748/1748 - 2s - loss: 0.6540 - acc: 0.7162
Epoch 13/100
1748/1748 - 2s - loss: 0.6471 - acc: 0.7185
Epoch 14/100
1748/1748 - 2s - loss: 0.6500 - acc: 0.7145
Epoch 15/100
1748/1748 - 2s - loss: 0.6365 - acc: 0.7214
Epoch 16/100
1748/1748 - 2s - loss: 0.6223 - acc: 0.7374
Epoch 17/100
1748/1748 - 2s - loss: 0.6133 - acc: 0.7374
Epoch 18/100
1748/1748 - 2s - loss: 0.60

In [18]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Test Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
#print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
#print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

1748/1748 - 2s - loss: 0.2507 - acc: 0.9113
Test Neural Network - Loss: 0.2507263141038489, Accuracy: 0.9113272428512573


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [24]:
# Create the GridSearchCV model
# Support vector machine linear classifier
#from sklearn.svm import SVC 
#model2 = SVC(kernel='linear')

In [35]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10,50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model2, param_grid, verbose=3, cv = 5)
grid

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 5, 10, 50], 'gamma': [0.0001, 0.0005, 0.001, 0.005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [36]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... C=1, gamma=0.0001, score=0.6784015223596574, total=   1.6s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] ....... C=1, gamma=0.0001, score=0.674618320610687, total=   1.6s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.3s remaining:    0.0s


[CV] ....... C=1, gamma=0.0001, score=0.648854961832061, total=   1.5s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...... C=1, gamma=0.0001, score=0.6545801526717557, total=   1.6s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...... C=1, gamma=0.0001, score=0.6717557251908397, total=   1.6s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...... C=1, gamma=0.0005, score=0.6784015223596574, total=   1.6s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ....... C=1, gamma=0.0005, score=0.674618320610687, total=   1.6s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ....... C=1, gamma=0.0005, score=0.648854961832061, total=   1.5s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...... C=1, gamma=0.0005, score=0.6545801526717557, total=   1.6s
[CV] C=1, gamma=0.0005 ...............................................
[CV] .

[CV] ...... C=50, gamma=0.0001, score=0.686964795432921, total=  13.7s
[CV] C=50, gamma=0.0001 ..............................................
[CV] ..... C=50, gamma=0.0001, score=0.6822519083969466, total=  15.0s
[CV] C=50, gamma=0.0001 ..............................................
[CV] ..... C=50, gamma=0.0001, score=0.6593511450381679, total=  13.8s
[CV] C=50, gamma=0.0001 ..............................................
[CV] ..... C=50, gamma=0.0001, score=0.6593511450381679, total=  13.9s
[CV] C=50, gamma=0.0001 ..............................................
[CV] ..... C=50, gamma=0.0001, score=0.6717557251908397, total=  15.5s
[CV] C=50, gamma=0.0005 ..............................................
[CV] ...... C=50, gamma=0.0005, score=0.686964795432921, total=  13.7s
[CV] C=50, gamma=0.0005 ..............................................
[CV] ..... C=50, gamma=0.0005, score=0.6822519083969466, total=  14.6s
[CV] C=50, gamma=0.0005 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  8.8min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 5, 10, 50], 'gamma': [0.0001, 0.0005, 0.001, 0.005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [37]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.6719435437726492


# Save the Model

In [38]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'model2.sav'
#joblib.dump(model1, filename)